In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm as tqdm

### 导入数据

In [11]:
data = pd.read_csv('data1.csv',encoding='utf-8')
data.head(10)

,platform_open_id,goods_num,price,is_member,discount_fee,outer_id,take_at,status,payment,member_id,order_payment,total_fee,goods_id
0,101ZX1xyfG/omvnxQPYI8YPFhoi2hfl26RoAPgzb9VqwG0=,1,3650.0,0,300.0,302945-01,2020-01-01 01:35:43,6,3350.0,0,3350.0,3350.0,520399672992
1,爱012Pr5yntLUcTKU0MtsjPGOs2DEMds1FAQI7fgfrP3PMg=,1,2690.0,0,600.0,227605-01,2020-01-01 08:55:17,6,2090.0,0,2090.0,2090.0,548452717637
2,朱01i4Rmip4x6V5VS/UZPBzc+Nnn6lnZ7cacExIx+8aylCY=,1,3990.0,1,400.0,226403-01,2020-01-01 09:48:35,6,3590.0,616297,3590.0,3590.0,564779472639
3,001bRbDk61o6cYW1XuaH8XgLgKHM0CMIQafAKaTQc17qGM=,1,3290.0,0,0.0,NaN,2020-01-01 09:56:04,6,3290.0,0,3290.0,3290.0,609664493135
4,t014Au7WHWPPI0OeT5DX13SQ5q8qGekDqre2uZnHtcQpY8=,1,3590.0,1,500.0,282846-01,2020-01-01 10:36:50,6,3090.0,220404,3090.0,3090.0,589083659427
5,莲01LF6zIyY7EH3xuSvbfZo7ltnn6lnZ7cacExIx+8aylCY=,1,3990.0,0,400.0,226403-01,2020-01-01 10:42:48,501,3590.0,0,3590.0,3590.0,564779472639
6,粉01UbFDeKuioQohILQR+wOsNm7XJ0Oc9YsnYNWwOnr6RWo=,1,2990.0,0,0.0,337173-01,2020-01-01 11:57:13,6,2990.0,0,2990.0,2990.0,609198131898
7,t01IVpMxupm69b7U2+nsTMgJJq8qGekDqre2uZnHtcQpY8=,1,3990.0,0,0.0,332873-01,2020-01-01 12:38:02,501,3990.0,0,3990.0,3990.0,608739665918
8,j01p2WEcqpLB97pTrpXeOF3YGdHwSigfMMQPj0dkBAC8Vk=,1,12298.0,0,0.0,NaN,2020-01-01 13:21:40,101,12298.0,0,12298.0,12298.0,43696478043
9,橘01NRMB2FbMyTAioDgFAKlTAq3Y2wWCx2gxb9BRyrBfqBc=,1,2690.0,1,600.0,227605-01,2020-01-01 14:02:07,6,2090.0,220661,2090.0,2090.0,548452717637


### 简单分析

### 年月和日分析

In [12]:
date = data['take_at'].str.split(' ').str[0]
data['year-month-day']=date
data['year-month'] = date.str.split('-').str[0]+'-'+date.str.split('-').str[1]
data_last = data[['status','goods_num','year-month','is_member','discount_fee','price','payment']]
data_last=data_last[data_last['status']==6]
data_last['dis_scale']=(data_last['discount_fee']/data_last['price'])
# 去掉相对于价格折扣比例大于1的数据
data_last = data_last.drop(data_last[(data_last['dis_scale']>=1)].index)
data_last.head()

,status,goods_num,year-month,is_member,discount_fee,price,payment,dis_scale
0,6,1,2020-01,0,300.0,3650.0,3350.0,0.082192
1,6,1,2020-01,0,600.0,2690.0,2090.0,0.223048
2,6,1,2020-01,1,400.0,3990.0,3590.0,0.100251
3,6,1,2020-01,0,0.0,3290.0,3290.0,0.000000
4,6,1,2020-01,1,500.0,3590.0,3090.0,0.139276


In [13]:
# 看折扣比例情况情况
dis_scale = data_last.groupby('dis_scale')['dis_scale'].count()
dis_scale

dis_scale
0.000000    898986
0.006689         2
0.011111         1
0.019268         7
0.026525         1
             ...  
0.999667       244
0.999900     24121
0.999997         1
0.999998         1
0.999999      3045
Name: dis_scale, Length: 755, dtype: int64

In [20]:
# 看会员情况
is_member = data_last.groupby('is_member')
list(is_member)[1][1]

,status,goods_num,year-month,is_member,discount_fee,price,payment,dis_scale
2,6,1,2020-01,1,400.0,3990.00,3590.00,0.100251
4,6,1,2020-01,1,500.0,3590.00,3090.00,0.139276
9,6,1,2020-01,1,600.0,2690.00,2090.00,0.223048
33,6,1,2020-01,1,0.0,2990.00,2990.00,0.000000
44,6,1,2020-01,1,600.0,2690.00,2090.00,0.223048
...,...,...,...,...,...,...,...,...
2076725,6,1,2021-06,1,0.0,200.00,200.00,0.000000
2076731,6,1,2021-06,1,0.0,0.01,0.01,0.000000
2076732,6,1,2021-06,1,0.0,0.01,0.01,0.000000
2076739,6,1,2021-06,1,0.0,2990.00,2990.00,0.000000


In [15]:
# 看日期销量情况
date_= data_last.groupby('year-month')['year-month'].count()
date_

year-month
2020-01     52424
2020-02     41971
2020-03     49171
2020-04     54820
2020-05     77726
2020-06    139355
2020-07     23293
2020-08     32492
2020-09     30309
2020-10    216179
2020-11    197206
2020-12     53974
2021-01     51955
2021-02     41583
2021-03     67789
2021-04     48683
2021-05     93161
2021-06    141080
Name: year-month, dtype: int64

In [16]:
discount_= data_last.groupby('discount_fee')['discount_fee'].count()
discount_

discount_fee
0.00       898986
5.00            1
20.00           2
29.99         244
30.00          24
            ...  
3200.00         1
3500.00         4
3600.00         1
5489.99         1
9998.99      3045
Name: discount_fee, Length: 516, dtype: int64

In [36]:
def cos_sim(a, b):
    a_norm = np.linalg.norm(a)
    b_norm = np.linalg.norm(b)
    cos = np.dot(a,b)/(a_norm * b_norm)
    return cos

In [35]:
# list(is_member)[0][1]非会员   list(is_member)[1][1]会员订单
month_1 = list(is_member)[1][1]
data_month1 = month_1.groupby('year-month')
month_1 = list(data_month1)
discount_month1 = []
num_month = []
discount_scale_month1 = []
for i in tqdm(range(len(list(data_month1)))):
    gmv_1= month_1[i][1]['payment'].sum()
    num_1= month_1[i][1]['goods_num'].sum()
    discount_1 = month_1[i][1]['discount_fee'].sum()
    num_month.append(num_1)
    discount_month1.append(discount_1)
    discount_scale_month1.append(discount_1/gmv_1)
    
money_ = discount_month1/max(discount_month1)
scale_ = discount_scale_month1/max(discount_scale_month1)
num_ = num_month/max(num_month)
print(money_)
print(scale_ )
print(num_)
print(f'折扣钱与销量相关系数的值为：{cos_sim(money_,num_)}')
print(f'折扣比例与销量相关系数的值为：{cos_sim(scale_,num_)}')


[0.26317729 0.1080097  0.10520791 0.11583131 0.13264745 0.59703729
 0.81795461 1.         0.52803394 0.00906112 0.57750582 0.14624823
 0.15722542 0.09810968 0.1973014  0.14490107 0.09117106 0.54346783]
[0.10041032 0.04353943 0.05611823 0.06788768 0.05342467 0.11954549
 1.         0.83159165 0.58216643 0.0017845  0.10556959 0.08502497
 0.09112644 0.08368788 0.10531152 0.05628665 0.02938702 0.12229615]
[0.20596987 0.16470386 0.17402199 0.31388793 0.399834   0.69173427
 0.08511605 0.11784696 0.10871676 1.         0.70042597 0.16663012
 0.16807091 0.12397031 0.23495787 0.27436026 0.5708178  0.68442071]
折扣钱与销量相关系数的值为：0.5583473169351382
折扣比例与销量相关系数的值为：0.24888566143638158


In [19]:
# 画图
from pyecharts.charts import Line
from pyecharts import options as opts


month_list=pd.period_range('2020-01', periods=18, freq='M')
month_list = [str(month_list[i]) for i in range(len(month_list))]

def line_charts(data,x_name,y_name,title) -> Line:
    line = (
        Line()
        .add_xaxis(month_list)
        .add_yaxis(
            "折扣力度",
            data
        )
        .set_global_opts(title_opts=opts.TitleOpts(title=title)
                         ,yaxis_opts=opts.AxisOpts(name=y_name)
                         ,xaxis_opts=opts.AxisOpts(name=x_name),)
         .set_series_opts(
            label_opts=opts.LabelOpts(is_show=False), # 是否显示标签
        )
    )
    return line
# # 月的GMV
line_month = line_charts(discount_month1,x_name='month',y_name='member_discount',title='member_discount_analyse')
line_month.render("member_discount分析.html")

'E:\\py_\\out_work\\YJF_jp_work\\member_discount分析.html'